# 営業成約予測（分類） Model

## Data の読み込みから Data の確認まで

### Data 読み込み

In [1]:
# 共通事前処理

# 余分な Warning を非常にする
import warnings

warnings.filterwarnings('ignore')

# Library の import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# matplotlib 日本語化対応
import japanize_matplotlib

# DataFrame 表示用関数
from IPython.display import display

# 表示 Option の調整
# NumPy の浮動小数点の表示精度
np.set_printoptions(suppress=True, precision=4)
# pandas での浮動小数点の表示精度
pd.options.display.float_format = '{:.4f}'.format
# DataFrame ですべての項目を表示
pd.set_option('display.max_columns', None)
# Graph の Default Font 指定
plt.rcParams['font.size'] = 14
# 乱数の種
random_seed = 123

#### 公開 DataSet を DataFrame に読み込み

In [2]:
# 公開 Data の Download と解凍
# !wget https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip -O bank.zip | tail -n 1
# !unzip -o bank.zip | tail -n 1

In [3]:
# bank-full.csv を DataFrame に読み込み
df_all = pd.read_csv('bank-full.csv', sep=';') # 項目の区切り文字が Comma ではなく SemiCoron になっている為 `sep=';'` で区切り文字を指定。

In [4]:
# 項目名を日本語に置き換える
columns = ['年齢', '職業', '婚姻', '学歴', '債務不履行', '平均残高', '住宅_Loan', '個人_Loan', '連絡手段', '最終通話日', '最終通話月', '最終通話秒数', '通話回数_販促中', '前回販促後_経過日数', '通話回数_販促前', '前回販促結果', '今回販促結果']

df_all.columns = columns

#### DataFrame の内容確認

In [5]:
display(df_all.head())

,年齢,職業,婚姻,学歴,債務不履行,平均残高,住宅_Loan,個人_Loan,連絡手段,最終通話日,最終通話月,最終通話秒数,通話回数_販促中,前回販促後_経過日数,通話回数_販促前,前回販促結果,今回販促結果
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


かなりの項目値が Text Data になっている。
機械学習 Model に入力できるような項目値が数値だけからなる Data に変換していく。

#### Data の確認

In [6]:
# 学習 Data の件数と項目数確認
print(df_all.shape)
print()

# 「今回販促結果」の値の分布確認
print(df_all['今回販促結果'].value_counts())
print()

# 営業成功率
rate = df_all['今回販促結果'].value_counts()['yes']/len(df_all)
print(f'営業成功率: {rate:.4f}')

(45211, 17)

no     39922
yes     5289
Name: 今回販促結果, dtype: int64

営業成功率: 0.1170


#### 欠損値の確認

In [7]:
# 欠損値の確認
print(df_all.isnull().sum())

年齢            0
職業            0
婚姻            0
学歴            0
債務不履行         0
平均残高          0
住宅_Loan       0
個人_Loan       0
連絡手段          0
最終通話日         0
最終通話月         0
最終通話秒数        0
通話回数_販促中      0
前回販促後_経過日数    0
通話回数_販促前      0
前回販促結果        0
今回販促結果        0
dtype: int64


欠損値は一切なく、対応不要。

## Data 前処理と Data 分割

### Data 前処理

#### 前処理 Step 1
Text の Label を持つ項目を One-Hot encoding する。
enc関数を利用して処理を進める。

In [8]:
# get_dummies 関数で Category 値を One-Hot encoding

# 項目を One-Hot encoding するための関数
def enc(df, column):
    df_dummy = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df.drop([column], axis=1), df_dummy], axis=1)
    return df


df_all2 = df_all.copy()
columns = ['職業', '婚姻', '学歴', '連絡手段', '前回販促結果']
for col in columns:
    df_all2 = enc(df_all2, col)

# 結果確認
df_all2.head()

,年齢,債務不履行,平均残高,住宅_Loan,個人_Loan,最終通話日,最終通話月,最終通話秒数,通話回数_販促中,前回販促後_経過日数,通話回数_販促前,今回販促結果,職業_admin.,職業_blue-collar,職業_entrepreneur,職業_housemaid,職業_management,職業_retired,職業_self-employed,職業_services,職業_student,職業_technician,職業_unemployed,職業_unknown,婚姻_divorced,婚姻_married,婚姻_single,学歴_primary,学歴_secondary,学歴_tertiary,学歴_unknown,連絡手段_cellular,連絡手段_telephone,連絡手段_unknown,前回販促結果_failure,前回販促結果_other,前回販促結果_success,前回販促結果_unknown
0,58,no,2143,yes,no,5,may,261,1,-1,0,no,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1
1,44,no,29,yes,no,5,may,151,1,-1,0,no,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1
2,33,no,2,yes,yes,5,may,76,1,-1,0,no,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1
3,47,no,1506,yes,no,5,may,92,1,-1,0,no,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1
4,33,no,1,no,no,5,may,198,1,-1,0,no,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1


One-Hot encoding した項目は、元の項目ごとに新しい項目ができ、値が１か０を取るようになっている。

#### 前処理 Step 2

In [9]:
# yes/no を 1/0 に置換

# ２値(yes/no)の値を（1/0)に置換する関数
def enc_bin(df, column):
    df[column] = df[column].map(dict(yes=1, no=0))
    return df

columns = ['債務不履行', '住宅_Loan', '個人_Loan', '今回販促結果']
for col in columns:
    df_all2 = enc_bin(df_all2, col)

# 結果確認
df_all2.head()

,年齢,債務不履行,平均残高,住宅_Loan,個人_Loan,最終通話日,最終通話月,最終通話秒数,通話回数_販促中,前回販促後_経過日数,通話回数_販促前,今回販促結果,職業_admin.,職業_blue-collar,職業_entrepreneur,職業_housemaid,職業_management,職業_retired,職業_self-employed,職業_services,職業_student,職業_technician,職業_unemployed,職業_unknown,婚姻_divorced,婚姻_married,婚姻_single,学歴_primary,学歴_secondary,学歴_tertiary,学歴_unknown,連絡手段_cellular,連絡手段_telephone,連絡手段_unknown,前回販促結果_failure,前回販促結果_other,前回販促結果_success,前回販促結果_unknown
0,58,0,2143,1,0,5,may,261,1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1
1,44,0,29,1,0,5,may,151,1,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1
2,33,0,2,1,1,5,may,76,1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1
3,47,0,1506,1,0,5,may,92,1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1
4,33,0,1,0,0,5,may,198,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1


#### 前処理 Step 3

最終通話月も One-Hot encoding すべきだが項目数が12個、増えてしまう。
月の間の数字は連続性で表現できるので数値１項目で表現する

In [11]:
# 月（jan, feb...）を1, 2... に置換

month_dict = dict(jan=1, feb=2, mar=3, apr=4, may=5, jun=6, jul=7, aug=8, sep=9, oct=10, nov=11, dec=12)

def enc_month(df, column):
    df[column] = df[column].map(month_dict)
    return df

df_all2 = enc_month(df_all2, '最終通話月')

# 結果確認
df_all2.head()

,年齢,債務不履行,平均残高,住宅_Loan,個人_Loan,最終通話日,最終通話月,最終通話秒数,通話回数_販促中,前回販促後_経過日数,通話回数_販促前,今回販促結果,職業_admin.,職業_blue-collar,職業_entrepreneur,職業_housemaid,職業_management,職業_retired,職業_self-employed,職業_services,職業_student,職業_technician,職業_unemployed,職業_unknown,婚姻_divorced,婚姻_married,婚姻_single,学歴_primary,学歴_secondary,学歴_tertiary,学歴_unknown,連絡手段_cellular,連絡手段_telephone,連絡手段_unknown,前回販促結果_failure,前回販促結果_other,前回販促結果_success,前回販促結果_unknown
0,58,0,2143,1,0,5,NaN,261,1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1
1,44,0,29,1,0,5,NaN,151,1,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1
2,33,0,2,1,1,5,NaN,76,1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1
3,47,0,1506,1,0,5,NaN,92,1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1
4,33,0,1,0,0,5,NaN,198,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1


すべての項目が数値に。＝機械学習 Model の入力 Data ができる形